## Initialization
To initialize the dataset class, we run the code below. We can change the dataroot parameter if the dataset is installed in a different folder. We can also omit it to use the default setup. These will be useful further below.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from nuimages import NuImages

ModuleNotFoundError: ignored

In [ ]:
nuim = NuImages(dataroot='C:/Users/mgavilan/Downloads/datasets/', version='v1.0-mini', verbose=True, lazy=True)

In [ ]:
nuim.category[2]

In [ ]:
nuim.table_names

In [ ]:
sample_idx = 10
sample = nuim.sample[sample_idx]
sample

In [ ]:
sample = nuim.get('sample', sample['token'])
sample

In [ ]:
sample_idx_check = nuim.getind('sample', sample['token'])
assert sample_idx == sample_idx_check

In [ ]:
key_camera_token = sample['key_camera_token']
print(key_camera_token)

In [ ]:
nuim.render_image(key_camera_token, annotation_type='all',
                  with_category=True, with_attributes=True, box_line_width=-1, render_scale=5)

In [ ]:
import os
counter = 0
#there are 50 samples for now, take one
for sample in nuim.sample:
  #generate list of object + surface tokens for a given sample image
  object_tokens, surface_tokens = nuim.list_anns(sample['token'], verbose=False)

  #iterate through object tokens, and collect attributes associated with each object
  o_we_want = []

  for o in object_tokens:
    att_names = []
    for att_token in nuim.get('object_ann', o)['attribute_tokens']:
      att_names.append(nuim.get('attribute', att_token)['name'])

    #if any object has a 'cycle.with_rider' or 'pedestrian' attribute, it means this image contains objects we care about detecting...people!!
    for a in att_names:
      if "cycle.with_rider" in a or "pedestrian" in a:
        o_we_want.append(o) #only keep object tokens that may contain a cyclist or pedestrian attribute

  print (o_we_want)

  #non-empty list means image contains something we care about detecting
  if o_we_want:
    #get the key_camera_token from the sample
    #render the image and save to our folder path
    key_cam_token = sample['key_camera_token']
    filename_path = 'C:/Users/mgavilan/Downloads/datasets/test/'
    filename_img = 'test' + str(counter)
    full_filename_img = filename_path + filename_img
    filename_xml = 'C:/Users/mgavilan/Downloads/datasets/test/test' + str(counter) + '.xml'
    nuim.render_image(key_cam_token, annotation_type='all', out_path = full_filename_img)

    #CONVERT IMAGES TO JPG

    #importing required packages and library
    import cv2

    # Loading .png image
    png_img = cv2.imread(full_filename_img + '.png')

    # converting to jpg file
    #saving the jpg file
    cv2.imwrite(full_filename_img + '.jpg', png_img, [int(cv2.IMWRITE_JPEG_QUALITY), 100])

    counter = counter + 1
    image_width = 1600
    image_height = 900


    ## WRITE THE PASCAL VOC XML ANNOTATION FILES

    with open(filename_xml, 'w') as f:
            f.write('<annotation>\n')
            f.write('\t<folder>XML</folder>\n')
            f.write('\t<filename>' + filename_img + '.jpg' + '</filename>\n')
            f.write('\t<path>' + os.getcwd() + os.sep + filename_img + '.jpg' + '</path>\n')
            f.write('\t<source>\n')
            f.write('\t\t<database>Unknown</database>\n')
            f.write('\t</source>\n')
            f.write('\t<size>\n')
            f.write('\t\t<width>' + str(image_width) + '</width>\n')
            f.write('\t\t<height>' + str(image_height) + '</height>\n')
            f.write('\t\t<depth>3</depth>\n') # assuming a 3 channel color image (RGB)
            f.write('\t</size>\n')
            f.write('\t<segmented>0</segmented>\n')


            #THE CODE BELOW ONLY ADDS 1 ANNOTATION FOR TESTING PURPOSES
            #ADD CODE TO GET ALL THE ANNOTATIONS IN ONE IMAGE FILE

            #for item in ??????:

            object_name = nuim.category[2]['name']
            x_min = str(textann[0])
            y_min = str(textann[1])
            x_max = str(textann[2])
            y_max = str(textann[3])
            # write each object to the file
            f.write('\t<object>\n')
            f.write('\t\t<name>' + object_name + '</name>\n')
            f.write('\t\t<pose>Unspecified</pose>\n')
            f.write('\t\t<truncated>0</truncated>\n')
            f.write('\t\t<difficult>0</difficult>\n')
            f.write('\t\t<bndbox>\n')
            f.write('\t\t\t<xmin>' + x_min + '</xmin>\n')
            f.write('\t\t\t<ymin>' + y_min + '</ymin>\n')
            f.write('\t\t\t<xmax>' + x_max + '</xmax>\n')
            f.write('\t\t\t<ymax>' + y_max + '</ymax>\n')
            f.write('\t\t</bndbox>\n')
            f.write('\t</object>\n')

            # Close the annotation tag once all the objects have been written to the file
            f.write('</annotation>\n')
            f.close() # Close the file
